<a href="https://colab.research.google.com/github/jainprashant03/ML_projects/blob/Tutorials%40Udemy/ChatGPTLunarLander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (727 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../swig4

In [ ]:
import gymnasium as gym
import numpy as np
import random
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from gym.wrappers import RecordVideo

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the Lunar Lander environment
env = gym.make("LunarLander-v3")
#env.seed(42)

# Hyperparameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
learning_rate = 0.001
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
batch_size = 64
max_episodes = 1000
max_steps = 1000
memory_size = 100000

# Replay buffer
memory = deque(maxlen=memory_size)

# Neural network for Q-learning
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the networks
q_network = DQN(state_size, action_size).to(device)
target_network = DQN(state_size, action_size).to(device)
target_network.load_state_dict(q_network.state_dict())
target_network.eval()

optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)

# Helper function to sample from memory
 #Sample Method: To randomly select batch of experiences from the memory buffer
def sample(memory, batchSize):
  batch = random.sample(memory, batch_size)
  states, actions, rewards, next_states, dones = zip(*batch)
  return (torch.tensor(states, dtype=torch.float32).to(device),
          torch.tensor(actions, dtype=torch.int64).to(device),
          torch.tensor(rewards, dtype=torch.float32).to(device),
          torch.tensor(next_states, dtype=torch.float32).to(device),
          torch.tensor(dones, dtype=torch.float32).to(device))

# Training loop
episode_rewards = []
for episode in range(max_episodes):
    state = env.reset()
    total_reward = 0
    for step in range(max_steps):
        # Select action
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
            q_values = q_network(state_tensor)
            action = torch.argmax(q_values).item()

        # Take action in the environment
        next_state, reward, done, _,_ = env.step(action)
        memory.append((state, action, reward, next_state, done))

        state = next_state
        total_reward += reward

        # Train the network if enough samples are available
        if len(memory) >= batch_size:
            states, actions, rewards, next_states, dones = sample(memory, batch_size)

            # Compute Q-values and targets
            q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
            next_q_values = target_network(next_states).max(1)[0]
            targets = rewards + (gamma * next_q_values * (1 - dones))

            # Compute loss and backpropagate
            loss = nn.MSELoss()(q_values, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if done:
            break

    # Update epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    episode_rewards.append(total_reward)

    # Update target network
    if episode % 10 == 0:
        target_network.load_state_dict(q_network.state_dict())

    print(f"Episode {episode + 1}, Reward: {total_reward}, Epsilon: {epsilon:.3f}")

    if np.mean(episode_rewards[-100:]) >= 200:
        print(f"Solved in {episode + 1} episodes!")
        break

# Plot rewards
plt.plot(episode_rewards)
plt.xlabel("Episodes")
plt.ylabel("Reward")
plt.title("Training Rewards")
plt.show()

# Generate video of the trained agent
env = RecordVideo(env, './videos', episode_trigger=lambda x: True)
state = env.reset()
done = False
while not done:
    state_tensor = torch.tensor([state], dtype=torch.float32).to(device)
    q_values = q_network(state_tensor)
    action = torch.argmax(q_values).item()
    state, _, done, _ = env.step(action)
env.close()

print("Training complete! Video saved in './videos'.")


<ipython-input-10-a74a743d56a9>:61: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return (torch.tensor(states, dtype=torch.float32).to(device),


ValueError: expected sequence of length 8 at dim 1 (got 2)